Télécom MS Group

HI!CKATHON 2022  

6/3/2022  

# I.	OVERVIEW
## 1.	Project Background and Description
 	Describe how this project came about, and the purpose.
We are a team of 4 MS students learning about Big Data and AI this year at Télécom Paris. We heard about this project directly from our school's teacher and we saw it as a great opportunity to tackle a challenging task. For all of us it was a first-timer participating in such a competition. 

## 2.	Project Scope
 	 Scope answers questions including what will be done, what won’t be done, and what the result will look like.

What won't be done :
- 

## 3.	Presentation of the group
 	Include your specialization at school, etc.

| First name | Last name | Year of studies & Profile | School  | Skills       | Roles/Tasks | Observations |
| ---------- | --------- | ------------------------- | ------  | ------       | ----------- | ------------ |
| Philéas    | SAMIR     | MS Big Data - Psychologist| Télécom | Python/ML/DL | Modeling    |       |
| Pierrick   | LEROY     | MS AI                     | Télécom | Python/ML/DL | Modeling    |              |
| Laouaouda  | AMINE     | MS Big Data               | Télécom | Python/ML/DL | EDA         |              |
| Mejait     | ASMAE     | MS AI                     | Télécom | Python/ML/DL | EDA         |              |



## 4.	Task Management
 	Describe how you interacted and collaborated as a team, and the effect of every member’s unique background on the project.
We had various proficiency and expertise levels on the team as MS people at Télécom all have different backgrounds. To collaborate efficiently we set up a few things, notably:
- Common data exploration : to make sure everyone was on the same page we broadcasted

Pierrick 


# II.	PROJECT MANAGEMENT
## 1.	Data Understanding
 	Provided the initial collection of data has already occurred, this step includes identifying and defining the relevant data, exploring the range, scale, formats, contents, and biases of the data, and evaluating the quality and validity of the resulting data.
We have over 2600 images with less than half that contain a car. For the images with cars, on average, each model is represented around 13 times. Some models have as few as 4 images. This means we actually have little data to train a model image classifier.

## 2.	Data Pre-processing
 	Explain how the selection of data was manipulated and modified to remove redundant features and improve the quality of the data. Describe the preprocessing techniques used, such as data augmentation.
We tried to use information about the model to predict its emissions, such as the year or the model category (SUV, van, coupé, etc.). This did not provide good enough results to be pursued further.
Since we are working on images, preferred models are convolutional neural networks, also called ConvNets. These models, like all neural networks, can very easily overfit. To prevent such overfitting, we use data augmentation : we apply random transformations to images as they are passed to the network during training. This is a well-known technique that leads to better model generalization.

## 3.	Modeling Development
 	Describe how you selected algorithms, how you calibrated them according to the data and how - in fine - you selected the best AI model using a well-defined set of metrics.
Given the limited number of images per class, we chose to use two models stacked on top of eachother :
- a model based on the YOLO architecture and weights (transfer learning), to decide whether there is a car in the image, and get its bounding box,
- a regression model that predicts the car emissions from the YOLO-cropped image.

## 4.	 Deployment Strategy
 	What best practices/norms did you follow? How do you plan on deploying your IA solution?
Enter your answer here.


# III.	CARBON FOOTPRINT LIMITATION
 	Describe the taken measures/actions during the development of your solution in view of limiting the carbon footprint.
![Business_mdel]("./Business_model.PNG")


# IV.	CONCLUSION
 	Tell us about the actual results, their limitations as well as future perspectives and improvements.
Enter your answer here.

In [8]:
import torch
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import glob

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /home/jovyan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-3-5 torch 1.10.0+cu113 CUDA:0 (Tesla V100S-PCIE-32GB, 32510MiB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [23]:
crop = False

df = pd.DataFrame(columns = ["im_name","x_min","y_min","x_max","y_max", "e"])
for i, img in enumerate(glob.glob("../datasets/datasets_test/test/*")):
    row = [img.split("/")[-1]]
    im = np.asanyarray(Image.open(img))
    predictions = model(im).pandas().xyxy[0]
    predictions = predictions[((predictions["name"]=="car") | (predictions["name"]=="truck"))]
    try:
        running_box = predictions.loc[((predictions["xmax"]-predictions["xmin"]) * (predictions["ymax"] - predictions["ymin"])).argmax(), 
                               ["xmin","ymin","xmax","ymax"]].tolist()
        if crop:
            im_toPredict = im[running_box[1]:running_box[3],running_box[0]:running_box[2]]
            im_toPredict = cv2.resize(im_toPredict, dsize=(256,188), interpolation=cv2.INTER_CUBIC).T
            im_toPredict = np.expand_dims(im_toPredict, axis=0)
            # pred_e.append(model_emissions.predict(im_toPredict))
            e = make_prediction(model_emissions, im_toPredict)
    except:
        running_box = [0,0,0,0]
        if crop:
            e = 0
    
    if crop == False:
        im_toPredict = cv2.resize(im_toPredict, dsize=(256,188), interpolation=cv2.INTER_CUBIC).T
        im_toPredict = np.expand_dims(im_toPredict, axis=0)
        # pred_e.append(model_emissions.predict(im_toPredict))
        e = make_prediction(model_emissions, im_toPredict)
        
    row.extend(running_box)
    row.append(e)
    df.loc[i, :] = row

df

,im_name,x_min,y_min,x_max,y_max,e
0,85854523.jpg,191.033493,678.842102,1902.317993,1253.441528,0
1,16663988681.jpg,0,0,0,0,0
2,19090334369.jpg,0,0,0,0,0
3,71540198.jpg,70.247467,61.608627,1641.056152,809.442078,0
4,85427036.jpg,248.620361,197.806946,1038.264771,651.789429,0
...,...,...,...,...,...,...
1019,89862599.jpg,0.0,78.415718,989.538208,564.733154,0
1020,7055251597.jpg,0,0,0,0,0
1021,3848520911.jpg,0,0,0,0,0
1022,73537501.jpg,85.125076,81.848251,524.960022,368.802368,0


In [7]:
fig = plt.figure(figsize=(15,15))
i=1
for img_path in glob.glob("../datasets/datasets_test/test/*")[:9]:
    ax = fig.add_subplot(3,3,i)
    im = np.asanyarray(Image.open(img_path))
    predictions = model(im).pandas().xyxy[0]
    predictions = predictions[((predictions["name"]=="car") | (predictions["name"]=="truck"))]# & (predictions["confidence"]>=.6)]
    
    for pred in predictions.iterrows():
        x1, y1, x2, y2 = pred[1][["xmin","ymin","xmax","ymax"]]
        rect = patches.Rectangle(
            (x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor="blue", facecolor="none",
        )
        ax.add_patch(rect)
        ax.text(x1, y2+5, round(pred[1]["confidence"],2), va="top", fontsize=18)
    
    ax.imshow(im)
    ax.set_title(model_emissions.predict(im))
    ax.set_xticks([])
    ax.set_yticks([])
    i+=1

plt.savefig("test_preds.png")
plt.show()